In [5]:
"""
Script Name: Nonlinear Modulation Sequence Generator
Version ID: 1.2
Date: 2024-08-22
Author: JukkaTLinjama
File Name: placeholder_for_filename.py
Link to ChatGPT conversation: placeholder_for_link

Description:
This script generates a sequence based on amplitude and frequency modulation,
visualizes the sequence along with its modulation functions, and allows saving
the generated sequence to a CSV file with four significant digits. The sequence generator is modular,
with separate functions for amplitude and frequency modulation and final signal generation.

Libraries: numpy, matplotlib, pandas, ipywidgets

Usage:
1. Adjust the sliders to set your desired sequence parameters.
2. The sequence will be plotted automatically along with modulation values.
3. Click the "Save Sequence to CSV" button to save the current sequence to a CSV file.
"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, FloatSlider, Button, Output, Text
from IPython.display import display, clear_output

def amplitude_modulation(t, amp_mod_freq, modulation_index, amp_phase):
    """
    Nonlinear amplitude modulation function.

    Parameters:
    t (float): Time value.
    amp_mod_freq (float): Amplitude modulation frequency.
    modulation_index (float): Modulation index controlling the depth of modulation.
    amp_phase (float): Initial phase of amplitude modulation.

    Returns:
    float: The amplitude modulation value.
    """
    return np.power(1 + modulation_index * np.sin(2 * np.pi * amp_mod_freq * t + amp_phase), 3)

def frequency_modulation(t, freq_mod_freq, modulation_index, freq_phase, natural_frequency=0.5):
    """
    Nonlinear frequency modulation function.

    Parameters:
    t (float): Time value.
    freq_mod_freq (float): Frequency modulation frequency.
    modulation_index (float): Modulation index controlling the depth of modulation.
    freq_phase (float): Initial phase of frequency modulation.
    natural_frequency (float): Base natural frequency.

    Returns:
    float: The modulated frequency value.
    """
    freq_mod = np.sin(2 * np.pi * freq_mod_freq * t + freq_phase)
    return natural_frequency * np.power(1 + modulation_index * freq_mod, 3)

def generate_sequence(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, 
                      amp_phase=0.0, freq_phase=0.0, natural_frequency=0.5, amplitude=100):
    """
    Generates a sequence using amplitude and frequency modulation functions.
    """
    length = int(length)  # Ensure length is an integer
    sequence = []
    times = np.arange(length) * dt
    
    for i in range(length):
        time = times[i]
        
        # Modulation functions
        amp_mod_value = amplitude_modulation(time, amp_mod_freq, modulation_index, amp_phase)
        freq_mod_value = frequency_modulation(time, freq_mod_freq, modulation_index, freq_phase, natural_frequency)
        
        # Signal generation using the modulation values
        sine_wave = np.sin(2 * np.pi * freq_mod_value * time)
        value = amplitude * amp_mod_value * sine_wave
        
        # Storing the result and modulation values
        sequence.append({
            'value': value,
            'amplitude_modulation': amp_mod_value,
            'frequency_modulation': freq_mod_value
        })
    
    return sequence, times

def plot_sequence(sequence, times, title="Sequence"):
    values = [point['value'] for point in sequence]
    amplitude_modulations = [point['amplitude_modulation'] for point in sequence]
    frequency_modulations = [point['frequency_modulation'] for point in sequence]
    
    plt.figure(figsize=(14, 10))
    
    # Plot the main sequence
    plt.subplot(4, 1, 1)
    plt.plot(times, values, label="Main Sequence")
    plt.title(title)
    plt.xlabel("Time (s)")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()
    
    # Plot the amplitude modulation function
    plt.subplot(4, 1, 2)
    plt.plot(times, amplitude_modulations, color='orange', label="Amplitude Modulation")
    plt.title("Amplitude Modulation")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.legend()
    
    # Plot the frequency modulation function
    plt.subplot(4, 1, 3)
    plt.plot(times, frequency_modulations, color='green', label="Frequency Modulation")
    plt.title("Frequency Modulation")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")
    plt.grid(True)
    plt.legend()

    # Plot the actual modulated sine wave
    plt.subplot(4, 1, 4)
    plt.plot(times, np.sin(2 * np.pi * np.array(frequency_modulations) * times), label="Modulated Sine Wave")
    plt.title("Actual Modulated Sine Wave")
    plt.xlabel("Time (s)")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Define sliders for interactive control
length_slider = FloatSlider(min=100, max=1000, step=10, value=500, description="Length:")
dt_slider = FloatSlider(min=0.01, max=0.1, step=0.01, value=0.02, description="dt:")
amp_mod_freq_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description="Amp Mod Freq:")
freq_mod_freq_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description="Freq Mod Freq:")
modulation_index_slider = FloatSlider(min=0.0, max=0.5, step=0.01, value=0.25, description="Modulation Index:")
amp_phase_slider = FloatSlider(min=0.0, max=2*np.pi, step=0.1, value=0.0, description="Amp Phase:")
freq_phase_slider = FloatSlider(min=0.0, max=2*np.pi, step=0.1, value=0.0, description="Freq Phase:")

# Output area for plotting
output = Output()

def update_plot(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, amp_phase, freq_phase):
    with output:
        clear_output(wait=True)
        sequence, times = generate_sequence(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, amp_phase, freq_phase)
        plot_sequence(sequence, times, title="Generated Sequence")
        return sequence

# Create the interactive plot
interactive_plot = interact(update_plot, 
                            length=length_slider, 
                            dt=dt_slider, 
                            amp_mod_freq=amp_mod_freq_slider, 
                            freq_mod_freq=freq_mod_freq_slider, 
                            modulation_index=modulation_index_slider,
                            amp_phase=amp_phase_slider,
                            freq_phase=freq_phase_slider)
display(output)

def save_sequence_to_csv(sequence, filename="target_sequence.csv"):
    """
    Saves the generated sequence to a CSV file with four significant digits.
    """
    data = [{'Value': round(point['value'], 4), 
             'Amplitude_Modulation': round(point['amplitude_modulation'], 4), 
             'Frequency_Modulation': round(point['frequency_modulation'], 4)} 
            for point in sequence]
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Sequence saved to {filename} with four significant digits.")

# Add a text box for filename prompt and a button to save the sequence
filename_textbox = Text(value="target_sequence.csv", description="File name:")
save_button = Button(description="Save Sequence to CSV")

def on_save_button_clicked(b):
    sequence = update_plot(length_slider.value, dt_slider.value, 
                           amp_mod_freq_slider.value, freq_mod_freq_slider.value, 
                           modulation_index_slider.value, amp_phase_slider.value, freq_phase_slider.value)
    
    # Use the entered filename or fall back to default
    filename = filename_textbox.value if filename_textbox.value.strip() != "" else "target_sequence.csv"
    save_sequence_to_csv(sequence, filename)

save_button.on_click(on_save_button_clicked)
display(filename_textbox, save_button)


interactive(children=(FloatSlider(value=500.0, description='Length:', max=1000.0, min=100.0, step=10.0), Float…

Output()

Text(value='target_sequence.csv', description='File name:')

Button(description='Save Sequence to CSV', style=ButtonStyle())

Sequence saved to target_sequence.csv with four significant digits.


# v 1.7 debugging freq modul..

In [15]:
"""
Script Name: Nonlinear Modulation Sequence Generator
Version ID: 1.7
Date: 2024-08-22
Author: JukkaTLinjama
File Name: placeholder_for_filename.py
Link to ChatGPT conversation: placeholder_for_link

Description:
This script generates a sequence based on amplitude and frequency modulation,
visualizes the sequence along with its modulation functions, and allows saving
the generated sequence to a CSV file with three significant digits. The sequence generator is modular,
with separate functions for amplitude and frequency modulation and final signal generation.

Libraries: numpy, matplotlib, pandas, ipywidgets

Usage:
1. Adjust the sliders to set your desired sequence parameters.
2. The sequence will be plotted automatically along with modulation values.
3. Click the "Save Sequence to CSV" button to save the current sequence to a CSV file.
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, FloatSlider, Button, Output, Text, Layout
from IPython.display import display, clear_output

def amplitude_modulation(t, amp_mod_freq, modulation_index, amp_phase):
    """
    Linear amplitude modulation function.
    """
    return 1 + modulation_index * np.sin(2 * np.pi * amp_mod_freq * t + amp_phase)

def frequency_modulation(t, freq_mod_freq, modulation_index, freq_phase):
    """
    Linear frequency modulation function.
    """
    freq_mod = np.sin(2 * np.pi * freq_mod_freq * t + freq_phase)
    return 1 + modulation_index * freq_mod

def generate_sequence(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, 
                      amp_phase=0.0, freq_phase=0.0, base_frequency=0.5, amplitude=100):
    """
    Generates a sequence using linear amplitude and frequency modulation functions.
    Also tracks frequency modulation values for debugging.
    """
    length = int(length)  # Ensure length is an integer
    sequence = []
    freq_mod_tracking = []  # Track frequency modulation values for debugging
    times = np.arange(length) * dt
    
    for i in range(length):
        time = times[i]
        
        # Modulation functions
        amp_mod_value = amplitude_modulation(time, amp_mod_freq, modulation_index, amp_phase)
        freq_mod_value = frequency_modulation(time, freq_mod_freq, modulation_index, freq_phase)
        
        # Track the frequency modulation value
        freq_mod_tracking.append(freq_mod_value)
        
        # Signal generation using the modulation values with base frequency in sine wave
        sine_wave = np.sin(2 * np.pi * base_frequency * freq_mod_value * time)
        value = amplitude * amp_mod_value * sine_wave
        
        # Storing the result and modulation values
        sequence.append({
            'value': value,
            'amplitude_modulation': amp_mod_value,
            'frequency_modulation': freq_mod_value
        })
    
    return sequence, times, freq_mod_tracking

def plot_sequence(sequence, times, base_frequency, dt, freq_mod_tracking, title="Sequence"):
    values = [point['value'] for point in sequence]
    amplitude_modulations = [point['amplitude_modulation'] for point in sequence]
    rms_value = np.sqrt(np.mean(np.square(values)))

    plt.figure(figsize=(14, 14))
    
    # Plot the main sequence
    plt.subplot(5, 1, 1)
    plt.plot(times, values, label="Main Sequence")
    plt.title(title)
    plt.xlabel("Time (s)")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()
    
    # Combined amplitude modulation and frequency modulation tracking
    plt.subplot(5, 1, 2)
    plt.plot(times, amplitude_modulations, color='orange', label="Amplitude Modulation")
    plt.plot(times, freq_mod_tracking, color='green', label="Frequency Modulation (Tracking)")
    plt.title("Amplitude and Frequency Modulation (Tracking)")
    plt.xlabel("Time (s)")
    plt.ylabel("Modulation Value")
    plt.grid(True)
    plt.legend()
    
    # Plot the amplitude histogram (smaller)
    plt.subplot(5, 1, 3)
    plt.hist(values, bins=50, color='blue', alpha=0.7)
    plt.title(f"Amplitude Histogram (RMS Value: {rms_value:.3f})")
    plt.xlabel("Amplitude")
    plt.ylabel("Frequency")
    plt.grid(True)

    # Plot the actual modulated sine wave
    plt.subplot(5, 1, 4)
    plt.plot(times, np.sin(2 * np.pi * base_frequency * np.array(freq_mod_tracking) * times), label="Modulated Sine Wave")
    plt.title("Actual Modulated Sine Wave")
    plt.xlabel("Time (s)")
    plt.ylabel("Value")
    plt.grid(True)
    plt.legend()

    # Frequency analysis (magnitude spectrum)
    plt.subplot(5, 1, 5)
    n = len(values)
    freqs = np.fft.fftfreq(n, d=dt)
    magnitudes = np.abs(np.fft.fft(values))
    plt.plot(freqs[:n//2], magnitudes[:n//2], label="Magnitude Spectrum")
    plt.title("Frequency Analysis (Magnitude Spectrum)")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.grid(True)
    plt.legend()

    # Plot frequency modulation tracking for debugging
    plt.figure(figsize=(10, 4))
    plt.plot(times, freq_mod_tracking, color='red', label="Frequency Modulation Tracking")
    plt.title("Frequency Modulation Tracking")
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency Modulation Value")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Add the version ID, date, and modulation calculation math as a small text
    plt.figtext(0.5, 0.01, f"Version ID: 1.7 | Date: 2024-08-22 | Modulation Calculation: "
                          "Amplitude = 1 + modulation_index * sin(2π * amp_mod_freq * t + amp_phase), "
                          "Frequency = base_frequency * (1 + modulation_index * sin(2π * freq_mod_freq * t + freq_phase))",
                ha="center", fontsize=8, wrap=True)
    
    plt.tight_layout()
    plt.show()

# Define sliders for interactive control with wider layout
slider_layout = Layout(width='500px')
length_slider = FloatSlider(min=100, max=1000, step=10, value=500, description="Length:", layout=slider_layout)
dt_slider = FloatSlider(min=0.01, max=0.1, step=0.01, value=0.02, description="dt:", layout=slider_layout)
amp_mod_freq_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description="Amp Mod Freq:", layout=slider_layout)
freq_mod_freq_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description="Freq Mod Freq:", layout=slider_layout)
modulation_index_slider = FloatSlider(min=0.0, max=0.5, step=0.01, value=0.25, description="Modulation Index:", layout=slider_layout)
amp_phase_slider = FloatSlider(min=0.0, max=2*np.pi, step=0.1, value=0.0, description="Amp Phase:", layout=slider_layout)
freq_phase_slider = FloatSlider(min=0.0, max=2*np.pi, step=0.1, value=0.0, description="Freq Phase:", layout=slider_layout)
base_frequency_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description="Base Frequency:", layout=slider_layout)

# Output area for plotting
output = Output()

def update_plot(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, amp_phase, freq_phase, base_frequency):
    with output:
        clear_output(wait=True)
        sequence, times, freq_mod_tracking = generate_sequence(length, dt, amp_mod_freq, freq_mod_freq, modulation_index, amp_phase, freq_phase, base_frequency)
        plot_sequence(sequence, times, base_frequency, dt, freq_mod_tracking, title="Generated Sequence")
        return sequence

# Create the interactive plot
interactive_plot = interact(update_plot, 
                            length=length_slider, 
                            dt=dt_slider, 
                            amp_mod_freq=amp_mod_freq_slider, 
                            freq_mod_freq=freq_mod_freq_slider, 
                            modulation_index=modulation_index_slider,
                            amp_phase=amp_phase_slider,
                            freq_phase=freq_phase_slider,
                            base_frequency=base_frequency_slider)
display(output)

def save_sequence_to_csv(sequence, filename="target_sequence.csv"):
    """
    Saves the generated sequence to a CSV file with three significant digits.
    """
    data = [{'Value': round(point['value'], 3), 
             'Amplitude_Modulation': round(point['amplitude_modulation'], 3), 
             'Frequency_Modulation': round(point['frequency_modulation'], 3)} 
            for point in sequence]
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Sequence saved to {filename} with three significant digits.")

# Add a text box for filename prompt and a button to save the sequence
filename_textbox = Text(value="target_sequence.csv", description="File name:", layout=Layout(width='300px'))
save_button = Button(description="Save Sequence to CSV")

def on_save_button_clicked(b):
    sequence = update_plot(length_slider.value, dt_slider.value, 
                           amp_mod_freq_slider.value, freq_mod_freq_slider.value, 
                           modulation_index_slider.value, amp_phase_slider.value, freq_phase_slider.value, 
                           base_frequency_slider.value)
    
    # Use the entered filename or fall back to default
    filename = filename_textbox.value if filename_textbox.value.strip() != "" else "target_sequence.csv"
    save_sequence_to_csv(sequence, filename)

save_button.on_click(on_save_button_clicked)
display(filename_textbox, save_button)


interactive(children=(FloatSlider(value=500.0, description='Length:', layout=Layout(width='500px'), max=1000.0…

Output()

Text(value='target_sequence.csv', description='File name:', layout=Layout(width='300px'))

Button(description='Save Sequence to CSV', style=ButtonStyle())